# Homework 5

## Problem 1

SQP

In [1]:
#pip install qpsolvers[open_source_solvers]

import torch as t

#Initializing x
x = t.tensor([[1.], [1.]]) # x[0] = x1 and x[1] = x2

#Initializing mu and W just to get calculation started, calculated with initial conditions
u = t.tensor([[6.], [2.]]) # u[0] = u1 and u[1] = u2
W = t.tensor([[2., 0.], [0., (2 + 2*u[0] + 2*u[1])]])

#Calculating Lagrange Gradient
gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  )

e = 0.003
while t.norm(gL) > e:
    #Solve for s(k), u(k+1) using QP
    s = t.tensor([[1.], [1.]]) #for now
    
    #Line search, alpha(k), merit function and armijo line search
    alpha = 1 #for now
    
    #x(k+1) = x(k) + alpha(k)*s(k)
    x_dum = x + alpha*s
    x = x_dum
    
    #W(k+1), BFGS
    
    
    #Calculate Lagrange Gradient
    gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  ) #need to update x and mu above


KeyboardInterrupt: 

## Problem 2

moon lander